In [1]:
import pandas as pd
import numpy as np

from warehouse_modeling.induced_backorder_cost import *
from warehouse_modeling.lead_time_approximation import *
from warehouse_modeling.warehouse_optimization import *
from warehouse_modeling.warehouse_demand_modeling import *

from single_echelon_utils.inventory_level_computation import *
from single_echelon_utils.service_level_computation import *
from single_echelon_utils.dealer_optimization import *

from utils import *

## INDATA
First, read indata from a specified excel file and sheet.


In [2]:
excel_path = "/Users/AlexanderLarsson/documents/VSCode/test_indata.xlsx"
indata_sheet = "test_case_4_whNBD_Normal"
indataDF = pd.read_excel(excel_path,indata_sheet)
outdataDF = indataDF.copy()
indataDF

,Installation id,Type,Name,Transport time,Q,Holding cost,Target item fill rate,Demand type,Demand mean,Demand stdev,Reorder point
0,1,RDC,Johannesburg,10,40,1,NaN,NaN,NaN,NaN,100
1,2,Dealer,Deal1,10,10,1,0.95,Normal,1.0,4.0,28
2,3,Dealer,Deal2,3,10,1,0.95,Normal,1.0,4.0,12
3,4,Dealer,Deal3,2,10,1,0.95,Normal,1.0,4.0,10
4,5,Dealer,Deal4,1,10,1,0.95,Normal,1.0,4.0,7
5,6,Dealer,Deal5,4,10,1,0.95,Normal,1.0,4.0,15
6,7,Dealer,Deal6,3,10,1,0.95,Normal,1.0,4.0,12
7,8,Dealer,Deal7,2,10,1,0.95,Normal,1.0,4.0,10
8,9,Dealer,Deal8,5,10,1,0.95,Normal,1.0,4.0,17
9,10,Dealer,Deal9,4,10,1,0.95,Normal,1.0,4.0,15


In [3]:
Q_dealer_arr = indataDF.get(indataDF["Type"] == "Dealer").get("Q").to_numpy()
Q_warehouse = int(indataDF.get(indataDF["Type"] == "RDC").get("Q").to_numpy())
mu_dealer_arr = indataDF.get(indataDF["Type"] == "Dealer").get("Demand mean").to_numpy()
sigma_dealer_arr = indataDF.get(indataDF["Type"] == "Dealer").get("Demand stdev").to_numpy()
demand_type_arr = indataDF.get(indataDF["Type"] == "Dealer").get("Demand type").to_numpy()
demand_mean_arr = indataDF.get(indataDF["Type"] == "Dealer").get("Demand mean").to_numpy()
demand_stdev_arr = indataDF.get(indataDF["Type"] == "Dealer").get("Demand stdev").to_numpy()
demand_dealer_variance_arr = demand_stdev_arr**2
Q_subbatch_size = find_smallest_divisor(Q_dealer_arr)
L_wh = float(indataDF.get(indataDF["Type"]=="RDC").get("Transport time"))
R_dealer_arr = indataDF.get(indataDF["Type"] == "Dealer").get("Reorder point").to_numpy()
R_warehouse = int(indataDF.get(indataDF["Type"] == "RDC").get("Reorder point").to_numpy())


Calculating lead time demand mean and std dev for RDC

In [4]:
rdc_f_u_probability_array, wh_dist, mu_L, sigma2_L = warehouse_subbatch_demand_probability_array(Q_dealer_arr, mu_dealer_arr, 
    sigma_dealer_arr, demand_type_arr, L_wh, Q_subbatch_size)

outdataDF.loc[outdataDF["Type"] == "RDC","Demand type"] = wh_dist
outdataDF.loc[outdataDF["Type"] == "RDC","Lead time demand mean"] = mu_L * Q_subbatch_size
outdataDF.loc[outdataDF["Type"] == "RDC","Lead time demand stdev"] = math.sqrt(sigma2_L) * Q_subbatch_size
demand_mean_warehouse = mu_L * Q_subbatch_size/L_wh
outdataDF.loc[outdataDF["Type"] == "RDC","Demand mean"] = demand_mean_warehouse
demand_stdev_warehouse = math.sqrt(sigma2_L * Q_subbatch_size/L_wh)
outdataDF.loc[outdataDF["Type"] == "RDC","Demand stdev"] = demand_stdev_warehouse
outdataDF.loc[outdataDF["Type"] == "RDC", "Demand variance"] = sigma2_L * Q_subbatch_size/L_wh

rdc_f_u_probability_array, wh_dist, mu_L, sigma2_L = warehouse_subbatch_demand_probability_array(Q_dealer_arr, mu_dealer_arr, 
    sigma_dealer_arr, demand_type_arr, L_wh, Q_subbatch_size)


outdataDF.loc[outdataDF["Type"] == "Dealer","Q, subbatches"] = Q_dealer_arr/Q_subbatch_size

outdataDF

,Installation id,Type,Name,Transport time,Q,Holding cost,Target item fill rate,Demand type,Demand mean,Demand stdev,Reorder point,Lead time demand mean,Lead time demand stdev,Demand variance,"Q, subbatches"
0,1,RDC,Johannesburg,10,40,1,NaN,NBD,10.0,3.511912,100,100.0,35.119115,12.333522,NaN
1,2,Dealer,Deal1,10,10,1,0.95,Normal,1.0,4.000000,28,NaN,NaN,NaN,1.0
2,3,Dealer,Deal2,3,10,1,0.95,Normal,1.0,4.000000,12,NaN,NaN,NaN,1.0
3,4,Dealer,Deal3,2,10,1,0.95,Normal,1.0,4.000000,10,NaN,NaN,NaN,1.0
4,5,Dealer,Deal4,1,10,1,0.95,Normal,1.0,4.000000,7,NaN,NaN,NaN,1.0
5,6,Dealer,Deal5,4,10,1,0.95,Normal,1.0,4.000000,15,NaN,NaN,NaN,1.0
6,7,Dealer,Deal6,3,10,1,0.95,Normal,1.0,4.000000,12,NaN,NaN,NaN,1.0
7,8,Dealer,Deal7,2,10,1,0.95,Normal,1.0,4.000000,10,NaN,NaN,NaN,1.0
8,9,Dealer,Deal8,5,10,1,0.95,Normal,1.0,4.000000,17,NaN,NaN,NaN,1.0
9,10,Dealer,Deal9,4,10,1,0.95,Normal,1.0,4.000000,15,NaN,NaN,NaN,1.0


Calculating lead times approximations for dealers

In [16]:
Q_warehouse = int(int(indataDF.get(indataDF["Type"] == "RDC").get("Q"))/Q_subbatch_size)
R_warehouse = int(int(indataDF.get(indataDF["Type"] == "RDC").get("Reorder point"))/Q_subbatch_size)

W = waiting_time(negative_inventory(Q_subbatch_size,Q_warehouse,R_warehouse,rdc_f_u_probability_array),L_wh,mu_L,Q_subbatch_size)

outdataDF.loc[outdataDF["Type"]== "Dealer", "Waiting time"] = W
L_dealer_arr = outdataDF.get(outdataDF["Type"]== "Dealer").get("Transport time").to_numpy() + W
outdataDF.loc[outdataDF["Type"] == "Dealer", "Lead time"] = L_dealer_arr
Ldemand_dealer_mean_arr = outdataDF.get(outdataDF["Type"]== "Dealer").get("Lead time").to_numpy()*outdataDF.get(outdataDF["Type"]== "Dealer").get("Demand mean").to_numpy()
outdataDF.loc[outdataDF["Type"] == "Dealer", "Lead time demand mean"] = Ldemand_dealer_mean_arr
sqrt_dealer_lead_time_arr = np.sqrt(L_dealer_arr)
print(sqrt_dealer_lead_time_arr)

Ldemand_dealer_stdev_arr = sqrt_dealer_lead_time_arr*demand_stdev_arr
print(Ldemand_dealer_stdev_arr)
print(demand_stdev_arr)
outdataDF.loc[outdataDF["Type"] == "Dealer", "Lead time demand stdev"] = Ldemand_dealer_stdev_arr
outdataDF

[3.25326897 1.89308188 1.60740755 1.25847488 2.14097151 1.89308188
 1.60740755 2.36299789 2.14097151 1.89308188]
[13.0130759   7.57232754  6.42963019  5.03389952  8.56388605  7.57232754
  6.42963019  9.45199155  8.56388605  7.57232754]
[4. 4. 4. 4. 4. 4. 4. 4. 4. 4.]


,Installation id,Type,Name,Transport time,Q,Holding cost,Target item fill rate,Demand type,Demand mean,Demand stdev,Reorder point,Lead time demand mean,Lead time demand stdev,Demand variance,"Q, subbatches",Waiting time,Lead time,Fill rate
0,1,RDC,Johannesburg,10,40,1,NaN,NBD,10.0,3.511912,100,100.000000,35.119115,12.333522,NaN,NaN,NaN,NaN
1,2,Dealer,Deal1,10,10,1,0.95,Normal,1.0,4.000000,28,10.583759,13.013076,NaN,1.0,0.583759,10.583759,0.519950
2,3,Dealer,Deal2,3,10,1,0.95,Normal,1.0,4.000000,12,3.583759,7.572328,NaN,1.0,0.583759,3.583759,0.602854
3,4,Dealer,Deal3,2,10,1,0.95,Normal,1.0,4.000000,10,2.583759,6.429630,NaN,1.0,0.583759,2.583759,0.678056
4,5,Dealer,Deal4,1,10,1,0.95,Normal,1.0,4.000000,7,1.583759,5.033900,NaN,1.0,0.583759,1.583759,0.840641
5,6,Dealer,Deal5,4,10,1,0.95,Normal,1.0,4.000000,15,4.583759,8.563886,NaN,1.0,0.583759,4.583759,0.572728
6,7,Dealer,Deal6,3,10,1,0.95,Normal,1.0,4.000000,12,3.583759,7.572328,NaN,1.0,0.583759,3.583759,0.602854
7,8,Dealer,Deal7,2,10,1,0.95,Normal,1.0,4.000000,10,2.583759,6.429630,NaN,1.0,0.583759,2.583759,0.678056
8,9,Dealer,Deal8,5,10,1,0.95,Normal,1.0,4.000000,17,5.583759,9.451992,NaN,1.0,0.583759,5.583759,0.552348
9,10,Dealer,Deal9,4,10,1,0.95,Normal,1.0,4.000000,15,4.583759,8.563886,NaN,1.0,0.583759,4.583759,0.572728


Calculating fill rate for RDC

In [6]:
#Need fill rate calculations for gamma in service_level_computations. Gör om och förvandla induced backorder cost till target fill rate.


#if wh_dist == "Normal":
    #outdataDF.loc[outdataDF["Type"]== "RDC", "Fill rate"] = fill_rate_normal_demand(R_warehouse, Q_warehouse, demand_mean_warehouse, demand_stdev_warehouse)
#elif wh_dist == "Gamma":
    #outdataDF.loc[outdataDF["Type"]== "RDC", "Fill rate"] = #add here
#elif wh_dist == "NBD":
    #outdataDF.loc[outdataDF["Type"]== "RDC", "Fill rate"] = fill_rate_compound_poisson_demand(demand_prob_arr_negative_binomial(L_wh, demand_mean_warehouse, demand_stdev_warehouse), IL_prob_array_discrete_positive(R_warehouse, Q_warehouse, demand_prob_arr_negative_binomial(L_wh, demand_mean_warehouse, demand_stdev_warehouse)))

#outdataDF


Calculating fill rates for dealers

In [17]:

fill_rate_dealer_arr = []

for i in range(0, len(demand_type_arr)):
    if demand_type_arr[i] == "Normal":
        fill_rate = fill_rate_normal_demand(R_dealer_arr[i], Q_dealer_arr[i], Ldemand_dealer_mean_arr[i], Ldemand_dealer_stdev_arr[i])
        fill_rate_dealer_arr.append(fill_rate)
        print(fill_rate)
    elif demand_type_arr[i] == "Poisson":
        demand_prob_arr = demand_prob_arr_poisson(L_dealer_arr[i], Ldemand_dealer_mean_arr[i])
        IL_prob = IL_prob_array_discrete_positive(R_dealer_arr[i], Q_dealer_arr[i], demand_prob_arr)
        fill_rate_dealer_arr.append(fill_rate_poisson_demand(IL_prob))
    elif demand_type_arr[i] == "NBD":
        print(f"dealer mean arr: {Ldemand_dealer_mean_arr[i]}")
        demand_size_prob_arr = demand_size_arr_logarithmic(demand_mean_arr[i], demand_stdev_arr[i]**2)
        demand_prob_array = demand_prob_arr_negative_binomial(L_dealer_arr[i], demand_mean_arr[i], demand_stdev_arr[i]**2)
        IL_Prob = IL_prob_array_discrete_positive(R_dealer_arr[i], Q_dealer_arr[i], demand_prob_array)
        fill_rate_NBD = fill_rate_compound_poisson_demand(demand_size_prob_arr, IL_Prob)
        fill_rate_dealer_arr.append(fill_rate_NBD)

outdataDF.loc[outdataDF["Type"]== "Dealer", "Fill rate"] = fill_rate_dealer_arr
outdataDF

Reorder point: 28
Q: 10
Ldemand mean: 10.583759021931753
Ldemand std: 13.013075898914447
L dealer array: [10.58375902  3.58375902  2.58375902  1.58375902  4.58375902  3.58375902
  2.58375902  5.58375902  4.58375902  3.58375902]
demand variance: [16. 16. 16. 16. 16. 16. 16. 16. 16. 16.]
Ldemand stddev correct: 13.013075898914447
0.9536853950175859
Reorder point: 12
Q: 10
Ldemand mean: 3.583759021931754
Ldemand std: 7.5723275385384685
L dealer array: [10.58375902  3.58375902  2.58375902  1.58375902  4.58375902  3.58375902
  2.58375902  5.58375902  4.58375902  3.58375902]
demand variance: [16. 16. 16. 16. 16. 16. 16. 16. 16. 16.]
Ldemand stddev correct: 7.5723275385384685
0.9510738387855522
Reorder point: 10
Q: 10
Ldemand mean: 2.583759021931754
Ldemand std: 6.429630187725268
L dealer array: [10.58375902  3.58375902  2.58375902  1.58375902  4.58375902  3.58375902
  2.58375902  5.58375902  4.58375902  3.58375902]
demand variance: [16. 16. 16. 16. 16. 16. 16. 16. 16. 16.]
Ldemand stddev cor

,Installation id,Type,Name,Transport time,Q,Holding cost,Target item fill rate,Demand type,Demand mean,Demand stdev,Reorder point,Lead time demand mean,Lead time demand stdev,Demand variance,"Q, subbatches",Waiting time,Lead time,Fill rate
0,1,RDC,Johannesburg,10,40,1,NaN,NBD,10.0,3.511912,100,100.000000,35.119115,12.333522,NaN,NaN,NaN,NaN
1,2,Dealer,Deal1,10,10,1,0.95,Normal,1.0,4.000000,28,10.583759,13.013076,NaN,1.0,0.583759,10.583759,0.953685
2,3,Dealer,Deal2,3,10,1,0.95,Normal,1.0,4.000000,12,3.583759,7.572328,NaN,1.0,0.583759,3.583759,0.951074
3,4,Dealer,Deal3,2,10,1,0.95,Normal,1.0,4.000000,10,2.583759,6.429630,NaN,1.0,0.583759,2.583759,0.961009
4,5,Dealer,Deal4,1,10,1,0.95,Normal,1.0,4.000000,7,1.583759,5.033900,NaN,1.0,0.583759,1.583759,0.963937
5,6,Dealer,Deal5,4,10,1,0.95,Normal,1.0,4.000000,15,4.583759,8.563886,NaN,1.0,0.583759,4.583759,0.955983
6,7,Dealer,Deal6,3,10,1,0.95,Normal,1.0,4.000000,12,3.583759,7.572328,NaN,1.0,0.583759,3.583759,0.951074
7,8,Dealer,Deal7,2,10,1,0.95,Normal,1.0,4.000000,10,2.583759,6.429630,NaN,1.0,0.583759,2.583759,0.961009
8,9,Dealer,Deal8,5,10,1,0.95,Normal,1.0,4.000000,17,5.583759,9.451992,NaN,1.0,0.583759,5.583759,0.951640
9,10,Dealer,Deal9,4,10,1,0.95,Normal,1.0,4.000000,15,4.583759,8.563886,NaN,1.0,0.583759,4.583759,0.955983
